In [ ]:
"""
데이터 구성 원리 이해
    -모델과학습또는평가의도에맞게데이터를구성하는방법을이해하고있나
"""

In [ ]:
""" 요약 """

In [ ]:
############################################################################
# LLM은 input x와 label y, x를 통해 그 다음에 올 토큰을 추론하는거라 1씩 shift 된다.
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

In [ ]:
for i in range(0, len(token_ids) - max_length, stride):
    # 입력 청크: 현재 위치(i)부터 max_length만큼 가져온다. 인덱스 잘 볼것.
    input_chunk = token_ids[i : i + max_length]
                
    # 타겟 청크: 입력보다 1칸 뒤의 위치(i+1)부터 가져온다. llm이니까
    # GPT는 '다음 단어'를 맞추는 모델이므로, 정답은 입력보다 한 칸씩 뒤로 밀려있어야 한다.
    target_chunk = token_ids[i + 1 : i + max_length + 1]
    
    self.input_ids.append(torch.tensor(input_chunk))
    self.target_ids.append(torch.tensor(target_chunk))

In [ ]:
def random_split(df, train_frac, validation_frac):
    """
    데이터셋을 학습(Train), 검증(Validation), 테스트(Test) 셋으로 분할하는 함수
    """
    # 전체 데이터 섞기
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    # 분할 지점(인덱스) 계산
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    # 데이터 분할
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

In [ ]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        # 모델이 지원하는 최대 길이(context_size)를 넘지 않도록 자름
        idx_cond = idx[:, -context_size:]

In [ ]:
def custom_collate_fn( batch,
    pad_token_id=50256, ignore_index=-100, allowed_max_length=None,
    device="cpu"
):
    """
    데이터 로더에서 배치를 만들 때 사용하는 커스텀 함수.
    가변 길이의 시퀀스를 배치의 최대 길이에 맞춰 패딩(padding)하고, 
    정답(target) 데이터에서 패딩 부분은 손실(loss) 계산에서 제외하도록 처리합니다.
    """
    # 배치 내에서 가장 긴 시퀀스 길이 계산 (패딩을 위해 +1 여유)
    batch_max_length = max(len(item)+1 for item in batch)
    
    # 입력(inputs)과 정답(targets) 리스트 준비
    inputs_lst, targets_lst = [], []
    
    for item in batch:
        new_item = item.copy()
        # 문장 끝 토큰 <|endoftext|> 추가
        new_item += [pad_token_id]
        
        # 가장 긴 길이에 맞춰 패딩 추가
        padded = new_item + [pad_token_id] * (batch_max_length - len(new_item))
        
        # 입력은 마지막 토큰 제외 (0 ~ n-1)
        inputs = torch.tensor(padded[:-1]) 
        # 정답은 첫 번째 토큰 제외 (1 ~ n) -> 다음 토큰 예측 문제이므로
        targets = torch.tensor(padded[1:]) 

        # 중요: 패딩 부분 마스킹 처리
        # 타겟에서 패딩 토큰인 부분은 ignore_index(-100)로 바꿔서 CrossEntropyLoss 계산 시 무시되게 함
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        
        # 패딩이 시작되는 지점 이후의 모든 타겟 값을 -100으로 설정
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index
        # indices[1:] → 첫 번째 pad 위치를 제외한 나머지 pad 위치 인덱스 
        # 위치의 targets 값을 ignore_index로 바꿉니다.
        
        # 선택적으로 최대 시퀀스 길이 제한 (메모리 관리 등 목적)
        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]
            targets = targets[:allowed_max_length]

        inputs_lst.append(inputs)
        targets_lst.append(targets)
    
    # 텐서로 변환 및 디바이스(GPU/CPU)로 이동
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor